### Attempt to do Monte Carlo integration in 1D and then in multiple dimensions

The 1D case is fast, but the multi-D one works but is much slower than scipy nquad. Needs more work

In [1]:
import numpy as np
from scipy.integrate import quad, nquad
import matplotlib.pyplot as plt

#### Functions to test

In [2]:
def f1(x):
    return (15*x**3 + 21*x**2 + 41*x + 3)**(1/4) * (np.exp(-0.5*x))

def f1_mod(X):
    '''Modified for use with anyD version'''
    return (15*X[0]**3 + 21*X[0]**2 + 41*X[0] + 3)**(1/4) * (np.exp(-0.5*X[0]))

def f2(x):
    return x**3 + x**2

def f2_mod(X):
    '''Modified for use with anyD version'''
    return X[0]**3 + X[0]**2

def f2_db(x, y):
    return x*y**2

def f2_db2(X):
    '''Modified for use with anyD version'''
    return X[0]*X[1]**2

#### Integration functions

In [3]:
def monte_carlo_integrate_1D(f,lims,n=1000,subset_number=10):
    
    a = lims[0]
    b = lims[1]
    
    subsets = np.arange(0,n+1,n/subset_number)
    steps = n/subset_number
    u = np.zeros(n)
    
    for i in range(subset_number):
        start, end  = int(subsets[i]), int(subsets[i+1])
        u[start:end] = np.random.uniform(low=i/subset_number,high=(i+1)/subset_number,size=end-start)
    
    np.random.shuffle(u)
    
    u_func = f(a+(b-a)*u)
    
    return ((b-a)/n)*u_func.sum()
    

def monte_carlo_integrate_anyD(f,lims,n=1000):
    
    if len(np.shape(lims)) == 1:
        lims = lims.reshape(1,-1)
        
    #independent variables
    random_samps = []
    V = 1
    for i in range(lims.shape[0]):
        
        a = lims[i,0] #lower lim
        b = lims[i,1] #upper lim
        V *= (b-a)
        v = np.random.uniform(low=a,high=b,size=n)
        random_samps.append(v)
        
    random_samps= np.array(random_samps)
    
    #dependent variable
    #randomly choose y values between min(f(x)) and max(f(x))
    y = np.random.uniform(low=min(min(f(random_samps)),0),high=max(f(random_samps)),size=n)
    #find the area 
    V = (max(y)-min(y))*V
    #find the fraction of y points that fall below f(x) anf multiply by total area
    result = V * sum(abs(y) < abs(f(random_samps)))/n
    
    return result
      

#### Testing

In [12]:
monte_carlo_integrate_anyD(f1_mod,np.array([0,4]))

5.865727440971112

In [13]:
quad(f1,a=0,b=4)

(5.767433490695931, 5.492382344683848e-10)

In [14]:
monte_carlo_integrate_1D(f1,np.array([0,4]))

5.770454780512381

In [4]:
%%timeit -n100
monte_carlo_integrate_anyD(f1_mod,np.array([0,4]))

3.12 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%%timeit -n100 
quad(f1,a=0,b=4)

368 µs ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit -n100
monte_carlo_integrate_1D(f1,np.array([0,4]))

253 µs ± 34.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit -n100
nquad(f2_db, [(0,1), (0,1)])

340 µs ± 90.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%%timeit -n100
monte_carlo_integrate_anyD(f2_db2,np.array([[0,1],[0,1]]))

2.93 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
